In [1]:
! pip install pmdarima

You should consider upgrading via the 'C:\Users\Ruben\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import pmdarima as pmd
import numpy as np
from pmdarima.model_selection import train_test_split

In [3]:
dataset = pd.read_csv('./data/serie-storiche-ecommerce-pulito-no-buchi-temporali.csv')
dataset = dataset.sort_values(by='data')
dataset = dataset[dataset.totale != 0.0]

In [10]:
dataset.head()

,Unnamed: 0,data,new_index,totale,settore,Anno,Mese,Giorno,Mese (n°)
12130,12130,2014-01-07,47.0,2483.39,Running,2014,Gennaio,7,1
3045,3045,2014-01-15,48.0,2164.92,Calcio,2014,Gennaio,15,1
6077,6077,2014-01-16,49.0,2947.32,Casual,2014,Gennaio,16,1
6078,6078,2014-01-17,50.0,1814.79,Casual,2014,Gennaio,17,1
3053,3053,2014-01-23,52.0,982.44,Calcio,2014,Gennaio,23,1


In [14]:
arima_dataset = dataset[[ "data","totale"]]

In [15]:
train, test = train_test_split(arima_dataset, train_size=0.8)


In [16]:
train.head()

,data,totale
12130,2014-01-07,2483.39
3045,2014-01-15,2164.92
6077,2014-01-16,2947.32
6078,2014-01-17,1814.79
3053,2014-01-23,982.44


In [17]:
arima_model = pmd.auto_arima(train.totale, start_p=0, start_q=0, test="adf", seasonal=False,trace=True)

Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=252616.120, Time=0.06 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=251413.026, Time=0.12 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=251840.805, Time=0.22 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=250436.097, Time=0.18 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=249064.662, Time=0.23 sec
 ARIMA(4,0,0)(0,0,0)[0]             : AIC=248537.953, Time=0.34 sec
 ARIMA(5,0,0)(0,0,0)[0]             : AIC=247806.917, Time=0.43 sec
 ARIMA(5,0,1)(0,0,0)[0]             : AIC=246633.485, Time=2.93 sec
 ARIMA(4,0,1)(0,0,0)[0]             : AIC=246750.357, Time=1.97 sec
 ARIMA(5,0,2)(0,0,0)[0]             : AIC=246570.596, Time=4.28 sec
 ARIMA(4,0,2)(0,0,0)[0]             : AIC=246590.063, Time=2.43 sec
 ARIMA(5,0,3)(0,0,0)[0]             : AIC=246421.518, Time=5.90 sec
 ARIMA(4,0,3)(0,0,0)[0]             : AIC=246540.601, Time=6.18 sec
 ARIMA(5,0,4)(0,0,0)[0]             : AIC=246395.870, Time=9.28 sec
 ARIM

In [18]:
test["ARIMA"] = arima_model.predict(n_periods=len(test.totale))
test.head(10)

,data,totale,ARIMA
8464,2020-07-30,8691.87,8610.586303
2402,2020-07-30,18639.34,10898.849504
14526,2020-07-30,4741.64,10878.095072
5433,2020-07-30,22688.36,9428.128030
5434,2020-07-31,21600.85,10934.413496
14527,2020-07-31,16479.89,10426.040289
11496,2020-07-31,7543.50,9614.014511
2403,2020-07-31,10286.42,11156.194113
11497,2020-08-01,1526.06,9830.644249
2404,2020-08-01,16703.66,10139.087892


In [27]:
def mean_absolute_percentage_error(y_true, y_pred): 

    y_true, y_pred = np.array(y_true), np.array(y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [28]:
print(mean_absolute_percentage_error(test["totale"], test["ARIMA"]))

95.56753562235959


In [22]:
import plotly.express as px
fig = px.line(test, x='data', y=['totale',"ARIMA"])
fig.show()